In [ ]:
import math
import numpy as np

In [ ]:
# Global Variables
RAW_POWER_COL = 0
MARKET_DISPATCH_COL = 1
OPENING_CAPACITY_COL = 2
CLOSING_CAPACITY_COL = 3

CHARGE_PRICE_COL = 0
DISCHARGE_PRICE_COL = 1

##############################################################
###################### HELPER FUNCTIONS ######################
##############################################################

def createZeroScenario(max_battery_capacity = 580, 
                       max_battery_power = 300, 
                       charge_efficiency = 0.9, 
                       discharge_efficiency = 0.9, 
                       charge_period = 1,
                       discharge_period = 1):
    '''
    A function to create a scenario given length of charge and
    discharge period from the battery specifications.
    
    Parameters
    ----------
        - max_battery_capacity : double
            Maximum capacity of the battery. (default = 580)
        - max_battery_power : double
            Maximum battery power to discharge or charge at time t. (default = 300)
        - charge_efficiency : double
            Charging efficiency of the battery. (default = 0.9)
        - discharge_efficiency : double
            Discharge efficiency of the battery. (default = 0.9)
        - charge_period : int
            Given charge period. (default = 1)
        - discharge_period : int
            Given discharge period. (default = 1)
    
    Returns:
        - scenario : list
            List of raw power, dispatch, opening and closing capacity.
    '''
    
    # Shorten variable names
    b_cap = max_battery_capacity
    b_power = max_battery_power
    c_eff = charge_efficiency
    d_eff = discharge_efficiency
    
    # Shorten global variables calling.
    rp_col = RAW_POWER_COL
    md_col = MARKET_DISPATCH_COL
    oc_col = OPENING_CAPACITY_COL
    cc_col = CLOSING_CAPACITY_COL
    
    # Maximum Dispatches
    max_charge_dispatch = b_power / 2
    max_discharge_dispatch = b_power / 2 * d_eff

    
    total_period = range(charge_period + discharge_period)
    
    state = [[0, 0, 0, 0] for i in total_period]
    
    if charge_period > discharge_period:
        ### DISCHARGE PERIOD ###
        # raw power[-1] = maximum battery power
        state[-1][rp_col] = b_power
        # market dispatch[-1] = maximum discharge dispatch
        state[-1][md_col] = max_discharge_dispatch
        # opening capacity[-1] = min(maximum battery capacity, maximum discharge dispatch / discharge efficiency)
        state[-1][oc_col] = min(b_cap, max_discharge_dispatch / d_eff)
        
        # Iterate backwards
        for t in range(1, discharge_period):
            # closing capacity[-t -1] = opening capacity[-t]
            state[-t -1][cc_col] = state[-t][oc_col]
            
            state[-t -1][oc_col] = min(b_cap, state[-t -1][cc_col] + max_discharge_dispatch / d_eff)
            state[-t -1][md_col] = min(max_discharge_dispatch, (state[-t -1][oc_col] - state[-t -1][cc_col]) * d_eff)
            state[-t -1][rp_col] = min(b_power, state[-t -1][md_col] * 2 / d_eff)
        
        ### CHARGE PERIOD ###
        for t in range(charge_period - 1, charge_period + discharge_period):
            state[-t -1][cc_col] = state[-t][oc_col]
            state[-t -1][oc_col] = max(0, state[-t -1][cc_col] - max_charge_dispatch * c_eff)
            state[-t -1][md_col] = -min(max_charge_dispatch, (state[-t -1][cc_col] - state[-t -1][oc_col]) / c_eff)
            state[-t -1][rp_col] = -min(b_power, -(state[-t -1][md_col] * 2))
        
    elif charge_period == discharge_period:
        state[0][rp_col] = -b_power
        state[0][md_col] = -max_charge_dispatch
        state[0][cc_col] = min(b_cap, max_charge_dispatch * c_eff)
        for t in range(1, charge_period):
            state[t][rp_col] = -b_power
            state[t][md_col] = -max_charge_dispatch
            state[t][oc_col] = state[t-1][cc_col]
            state[t][cc_col] = min(b_cap, state[t][oc_col] + max_charge_dispatch * c_eff)
        
        for t in range(charge_period, charge_period + discharge_period - 1):
            state[t][rp_col] = b_power
            state[t][md_col] = max_discharge_dispatch
            state[t][oc_col] = state[t-1][cc_col]
            state[t][cc_col] = max(0, state[t][oc_col] - max_discharge_dispatch / d_eff)
        
        state[-1][oc_col] = state[-2][cc_col]
        state[-1][md_col] = state[-1][oc_col] * d_eff
        state[-1][rp_col] = state[-1][md_col] * 2 / d_eff
    
    return np.array(state, dtype = np.double)

##############################################################

def createScenario(max_battery_capacity = 580, 
                   max_battery_power = 300, 
                   charge_efficiency = 0.9, 
                   discharge_efficiency = 0.9, 
                   start_capacity = 0,
                   end_capacity = 0):
    '''
    A function to create all scenario possible of raw power, market dispatch, 
    opening and closing capacity from the battery specifications.
    
    Parameters
    ----------
        - max_battery_capacity : double
            Maximum capacity of the battery. (default = 580)
        - max_battery_power : double
            Maximum battery power to discharge or charge at time t. (default = 300)
        - charge_efficiency : double
            Charging efficiency of the battery. (default = 0.9)
        - discharge_efficiency : double
            Discharge efficiency of the battery. (default = 0.9)
        - start_capacity : double
            Intended starting capacity of battery. (default = 0)
        - end_capacity : double
            Intended ending capacity of battery. (default = 0)
    
    Returns:
        - scenarios : list
            List of raw power, dispatch, opening and closing capacity of all possible 
            combinations of charging and discharging period.
    '''
    
    capacity = [[0, 0]]
    raw_power = []
    dispatch = []
    
    b_cap = max_battery_capacity
    b_power = max_battery_power
    c_eff = charge_efficiency
    d_eff = discharge_efficiency
    
    max_charge_dispatch = b_power / 2
    max_charge_period = math.ceil(b_cap / (max_charge_dispatch * c_eff))
    
    max_discharge_dispatch = b_power / 2 * d_eff
    max_discharge_period = math.ceil(b_cap / (max_discharge_dispatch / d_eff))
    
    ratio = max_charge_period / max_discharge_period
    
    scenarios = {}
    for discharge_s in range(1, max_discharge_period + 1):
        for charge_s in range(discharge_s, math.ceil(discharge_s * ratio) + 1):
            if start_capacity == 0 and end_capacity == 0:
                scenarios[charge_s, discharge_s] = createZeroScenario(max_battery_capacity = max_battery_capacity, 
                                                                      max_battery_power = max_battery_power, 
                                                                      charge_efficiency = charge_efficiency, 
                                                                      discharge_efficiency = discharge_efficiency, 
                                                                      charge_period = charge_s,
                                                                      discharge_period = discharge_s)
    return scenarios

##############################################################
    
def createProfitCondition(spot_prices,
                          scenarios,
                          marginal_loss_factor = 0.991):
    '''
    A function to create a the condition where charging and discharging ensures 
    profit greater than equal to 0.
    
    Parameters
    ----------
        - spot_prices : list
            List of current state of prices.
        - scenarios : dictionary
            dictionary of all possible charging and discharging period combinations
            of battery settings. With the charging and discharging state as its key
            and the optimal battery settings as its value.
        - marginal_loss_factor : double
            Marginal loss factor of revenue during charging and discharging. (default = 0.991)
            
    Returns
    -------
        - minimal_price : double
            Minimum price to ensure profit.
    '''
    
    # Shorten parameters
    mlf = marginal_loss_factor

    # Shorten global variables calling.
    rp_col = RAW_POWER_COL
    md_col = MARKET_DISPATCH_COL
    oc_col = OPENING_CAPACITY_COL
    cc_col = CLOSING_CAPACITY_COL
    
    charge_p_col = CHARGE_PRICE_COL
    discharge_p_col = DISCHARGE_PRICE_COL
    
    
    

In [ ]:
scenarios = createScenario(start_capacity = 0, end_capacity = 0)